In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data=pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
train_data.head()

In [ ]:
train_data.drop(['Descript','Resolution','Address'],inplace=True,axis=1)

In [ ]:
test_data=pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')
test_data.head()

In [ ]:
print(train_data['DayOfWeek'].unique())
print(test_data['DayOfWeek'].unique())

In [ ]:
train_data.head()

In [ ]:
test_data.drop('Address',axis=1,inplace=True)

In [ ]:
train_data['DayOfWeek'].replace(to_replace=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                            value=[i for i in range(0,7)],
                              inplace=True)
test_data['DayOfWeek'].replace(to_replace=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
                            value=[i for i in range(0,7)],
                              inplace=True)

In [ ]:
dummies_train=pd.get_dummies(train_data['PdDistrict'])
dummies_test=pd.get_dummies(test_data['PdDistrict'])
train_data=pd.concat([train_data,dummies_train],axis=1)
train_data.drop('PdDistrict',inplace=True,axis=1)
test_data=pd.concat([test_data,dummies_test],axis=1)
test_data.drop('PdDistrict',inplace=True,axis=1)

In [ ]:
train_data.head()

In [ ]:
ls1=list(train_data['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][:4:]
train_data['Year']=ls1 

ls1=list(train_data['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][5:7:]
train_data['Month']=ls1 

ls1=list(train_data['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][8:10:]
train_data['Day']=ls1 

ls1=list(train_data['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][11:-6:]
train_data['Hours']=ls1

ls1=list(train_data['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][-5:-3:]
train_data['Minutes']=ls1 

In [ ]:
ls1=list(test_data['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][:4:]
test_data['Year']=ls1 

ls1=list(test_data['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][5:7:]
test_data['Month']=ls1 

ls1=list(test_data['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][8:10:]
test_data['Day']=ls1 

ls1=list(test_data['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][11:-6:]
test_data['Hours']=ls1

ls1=list(test_data['Dates'])
for i in range(len(ls1)):
    ls1[i]=ls1[i][-5:-3:]
test_data['Minutes']=ls1 

In [ ]:
train_data.drop('Dates',axis=1,inplace=True)
test_data.drop('Dates',axis=1,inplace=True)

In [ ]:
train_data.drop('Category',axis=1).head()

In [ ]:
test_data.head()

In [ ]:
test_data.drop('Id',axis=1).columns==train_data.drop('Category',axis=1).columns

In [ ]:
y=pd.get_dummies(train_data['Category'])
y

In [ ]:
X=train_data.drop(['Category'],axis=1)

In [ ]:
X = X.astype(float)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
y_train.dtypes

In [ ]:
from tensorflow import keras
from keras.layers import Dense,Dropout
from keras.models import Sequential

In [ ]:
model=Sequential()
model.add(Dense(128,input_shape=(X.shape[1],)))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(39,activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train=model.fit(X_train,y_train, 
         batch_size=32,    #batch_size-how many training exaples to use before updating weights
         #nb_epoch=20,
         epochs=10,
         verbose=2,      #verbose- determines how detailed the output will be
         validation_data=(X_test,y_test))

In [ ]:
test=test_data.drop(['Id'],axis=1)
test=test.astype(float)
test.dtypes

In [ ]:
pred=model.predict(test)
pred

In [ ]:
m = np.max(pred, axis=1).reshape(-1, 1)
predicted = np.array((pred == m), dtype='int32')
predicted

In [ ]:
sample=pd.read_csv('/kaggle/input/sf-crime/sampleSubmission.csv.zip')


In [ ]:
test_data

In [ ]:
col_names = list(sample.columns)
col_names.remove('Id')


In [ ]:
df_submission = pd.DataFrame()
df_submission['Id']=test_data['Id']
for i, entry in enumerate(col_names):
    df_submission[entry] = predicted[:,i]

In [ ]:
df_submission

In [ ]:
df_submission.to_csv('Submission.csv',index=False)